## Preprocessed Pupil Data IPA Computing

The latest update: refer to PupilComputingPipeline for details and updates.

### Reference
Mayjor Referenced papaers:
1. The Index of Pupillary Activity: Measuring Cognitive Load vis-à-vis Task Difficulty with Pupil Oscillation,https://dl.acm.org/doi/pdf/10.1145/3173574.3173856.
2. The Low/High Index of Pupillary Activity,https://dl.acm.org/doi/pdf/10.1145/3313831.3376394.
3. ComputationalMR. https://github.com/eth-ait/ComputationalMR.

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numba import jit
import math, pywt
import xml.etree.ElementTree

### Read preprocessed data

In [2]:
%store -r write_file_path
%store -r SAMPLING_RATE
# read_file_path = '../Data/RawData/21-08-20-38/right3D_119Hz.csv' # Un-hash tag this line when want to dynamically compute a file's data.
read_file_path = write_file_path
data = pd.read_csv(read_file_path)
print('Data read from: ' + read_file_path)

Data read from: ../Data/PreprocessedData/24-08-16-20-stand-noread/left2D_84Hz.csv


In [3]:
df = data[:].copy()
df

,Unnamed: 0,Dia_Spline3_Base,Event
0,2,3.820658,default
1,3,3.781562,default
2,4,3.684363,default
3,5,3.680976,default
4,6,3.830007,default
...,...,...,...
3413,3415,-2.433498,sitting
3414,3416,-2.377617,sitting
3415,3417,-2.351246,sitting
3416,3418,-3.344560,sitting


### Normalize timestamps

In [4]:
## Normalize the timestamp into a even serial.
def normalize_timestamp(df, sr):
    runner = 0
    for i in range(len(df)):
        df.loc[i, ('Timestamp')] = runner / sr  # The sampling rate is allocated dynamically.
        runner += 1
    return df

In [5]:
df_nor_ts = df.copy()
df_nor_ts = normalize_timestamp(df=df_nor_ts)
df_nor_ts

,Unnamed: 0,Dia_Spline3_Base,Event,Timestamp
0,2,3.820658,default,0.000000
1,3,3.781562,default,0.011905
2,4,3.684363,default,0.023810
3,5,3.680976,default,0.035714
4,6,3.830007,default,0.047619
...,...,...,...,...
3413,3415,-2.433498,sitting,40.630952
3414,3416,-2.377617,sitting,40.642857
3415,3417,-2.351246,sitting,40.654762
3416,3418,-3.344560,sitting,40.666667


### Calculate Low/High Index of Pupillary Activity

In [6]:
## This algorithm is directly copied from work: The Low/High Index of Pupillary Activity,https://dl.acm.org/doi/pdf/10.1145/3313831.3376394.

# Find the maximum modulus.
def modmax(d):
    # Compute signal modulus.
    m = [0.0] * len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])

    # If value is larger than both neighbours , and strictly larger than either , then it is a local maximum.
    t = [0.0] * len(d)

    for i in range(len(d)):
        ll = m[i - 1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i + 1] if i < len(d) - 2 else m[i]

        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i] ** 2)
        else:
            t[i] = 0.0
    return t

# Calculate L/H IPA.
def calc_lhipa(d):
    """
    Here d is a dataframe.
    """
    # Initialize parameter.
    wavelet = 'sym16'
    
    # Find max decomposition level.
    w = pywt.Wavelet(wavelet)
    maxlevel = pywt.dwt_max_level(len(d), filter_len=w.dec_len)

    # Set high and low frequency band indeces.
    hif, lof = 1, int(maxlevel / 2)

    # Get detail coefficients of pupil diameter signal d.
    cD_H = pywt.downcoef('d', d['Dia_Spline3_Base'], wavelet, 'per', level=hif)
    cD_L = pywt.downcoef('d', d['Dia_Spline3_Base'], wavelet, 'per', level=lof)
#     cD_H = pywt.downcoef('d', d['Diameter'], wavelet, 'per', level=hif)
#     cD_L = pywt.downcoef('d', d['Diameter'], wavelet, 'per', level=lof)

    # Normalize by 1/ 2j.
    cD_H[:] = [x / math.sqrt(2 ** hif) for x in cD_H]
    cD_L[:] = [x / math.sqrt(2 ** lof) for x in cD_L]

    # Obtain the LH:HF ratio.
    cD_LH = cD_L
    for i in range(len(cD_L)):
        cD_LH[i] = cD_L[i] / cD_H[((2 ** lof) // (2 ** hif)) * i]   # Used a '//' instead of '/' to make sure the index is an integer.

    # Detect modulus maxima , see Duchowski et al. [15].
    cD_LHm = modmax(cD_LH)

    # Threshold using universal threshold λuniv = σˆ (2logn), where σˆ is the standard deviation of the noise.
    lambda_univ = np.std(cD_LHm) * math.sqrt(2.0*np.log2(len(cD_LHm)))
    cD_LHt = pywt.threshold(cD_LHm, lambda_univ, mode="less")

    # Get signal duration (in seconds).
    # tt = d[-1].timestamp() - d[0].timestamp()
#     tt = d[-1].timestamp - d[0].timestamp
    tt = d.loc[len(d)-1, ('Timestamp')] - d.loc[0, ('Timestamp')]

    # Compute LHIPA.
    ctr = 0
    for i in range(len(cD_LHt)):
        if math.fabs(cD_LHt[i]) > 0:
            ctr += 1
    LHIPA = float(ctr) / tt
    return LHIPA


In [7]:
lhipa = calc_lhipa(d=df_nor_ts)
lhipa = round(lhipa, 4)
lhipa

3.0975

In [8]:
# # Encapsulate the data processing pipeline in a class.  
# # Might need a class to create multiple instances when dealing with data from left/right & 2D/3D model once in the future.
# def process_data(): 
#     # Read data.
#     # Already read in the former part.
    
#     # Normalize the timestamp.
#     # Already normalized.
    
#     # Calculate the LHIPA.
#     # Already calculated.
    
#     return lhipa